In [4]:
#notebook to download the csv of edges and nodes of a given network
import os
import requests 
import time
import string
import networkx as nx
import itertools
import networkx as nx
import pandas as pd
import json
import re
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords

from nltk.corpus import wordnet as wn #importing it
from nltk.stem.wordnet import WordNetLemmatizer #importing wordnet lemmatizer
from nltk import pos_tag #part-of-speech-tagger
from collections import defaultdict #defaultdict returns default value for non-existant keys you try to  access based on the function you passed in the constructor
from google.colab import files

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
def extract_text(df):       #extract the text from the tweets and RT
                            #works ONLY on .csv file
  list_strings = []
  for index in range(len(df)):
    if index % 1000 == 0:
      print(str(index)+' / '+str(len(df)))
    text = df.loc[index]['text']                          #if it is nor trucated nor a RT  i take "text"
    string = -1
    if (df.loc[index,"truncated"] == True):                 #if it is trucated I take "extended_tweet"
        string = df.loc[index,"extended_tweet"]
    if type(df.loc[index,"retweeted_status"]) != float:     #if it is a RT I take retweeted_status
        string = df.loc[index,"retweeted_status"]
    if type(string) == str :
        if(re.search('full_text\':(.+?)https',string) != None):     #if I find "full_text"
          s = re.search('full_text\':(.+?)https',string).group(1)
        if(re.search('text\':(.+?)https',string)!= None):
          s = re.search('text\':(.+?)https',string).group(1)
        else: 
          continue
        list_strings.append(s)
        #print(s)         
    else:
      list_strings.append(text)
      #print(text)
      

  return list_strings

In [14]:
# Cleaning, lemmatising and pos tagging tweets

nltk.download('words')
WORDS = set(nltk.corpus.words.words()) #the last two lines serve to download the corpus of standard English language words
nltk.download('stopwords') #downloading stopwords
STOP_WORDS = set(nltk.corpus.stopwords.words("english")) #taking the stop words from English language
nltk.download('wordnet') #downloading wordnet
nltk.download('averaged_perceptron_tagger') #downloading tagger
tag_map = defaultdict(lambda : wn.NOUN) #here we define that wn.NOUN is the default value for the dict
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

def lemma_pos_cleaner(tweet):

    tweet = re.sub("@[A-Za-z0-9]+","",tweet) # remove mentions
    tweet = re.sub("#[A-Za-z0-9]+", "",tweet) # remove hashtags
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) # remove http links
    tweet = " ".join(tweet.split())
    tweet = str.lower(tweet) #to lowercase 
    tweet = re.sub("'"," ",tweet) # remove aphostrophe

    #basically we use pos_tag function on tokens that we get by applying wordpunct tokenization
    #to tweet (it separates all the words and symbols)
    #then we pass the token along with it's wordnet pos value that we get from the tag_map dictionary (noun, adjective, verb or adverb) to the lemma function (the WordNetLemmatizer())
    lemma_function = WordNetLemmatizer()
    tweet = " ".join(lemma_function.lemmatize(token, tag_map[tag[0]]) for token, tag in nltk.pos_tag(nltk.wordpunct_tokenize(tweet))) #lemmatize
  

    # francesco: I removed also all 2 letters words and added specific words, words that appears frequently but are discarded because they are not in the english language
    SPECIFIC_WORDS = ['virus', 'coronavirus', 'covid19', 'covid', 'trump', 'hubei', 'beijing', 'xinjiang', 'jinping', 'korea', 'xinhua', 'india', 'taiwan','johnson','singapore', 'africa', 'japanese', 'france', 'asian', 'australia', 'french', 'asia', 'leishenshan', 'british', 'qingdao', 'fauci', 'america',  'california', 'sichuan', 'malaysia', 'huawei','thailand', 'shandong', 'italy', 'philippines', 'germany', 'facebook', 'african', 'shenzhen', 'tokyo', 'russian','uygur', '5g', 'pompeo', 'vietnam', 'australian', 'cambodia', 'zhejiang', 'yunnan', 'guangdong', 'korean', 'iran', 'washington']
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) if (w in WORDS or w in SPECIFIC_WORDS) and len(w)>2 and w not in STOP_WORDS ) #remove stop words
   
    return tweet

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [7]:
def frequency_dictionary(df):
  unique_words = {}

  for row in df:
    for word in row.split():
      #if the word is encountered for the first time add to dict as key and set its value to 0
      unique_words.setdefault(word,0)
      #increase the value (i.e the count) of the word by 1 every time it is encountered
      unique_words[word] += 1

  return unique_words

In [211]:
period = '_MarchApril2021'  # _JanFeb2020, _MarchApril2021, _SeptOct2020

In [212]:
China = pd.read_csv('/content/China'+period+'.csv')
USA = pd.read_csv('/content/USA'+period+'.csv')
China_USA = pd.read_csv('/content/China&USA'+period+'.csv')

In [213]:
# number of tweets:
print('China: ', len(China))
print('USA: ', len(USA))
print('China&USA: ', len(China_USA))

China:  1376
USA:  3957
China&USA:  5333


In [215]:
text_China = extract_text(China)
text_USA = extract_text(USA)
text_China_USA = extract_text(China_USA)

0 / 1376
1000 / 1376
0 / 3957
1000 / 3957
2000 / 3957
3000 / 3957
0 / 5333
1000 / 5333
2000 / 5333
3000 / 5333
4000 / 5333
5000 / 5333


In [216]:
cleaned_text_China = [lemma_pos_cleaner(txt) for txt in text_China]
cleaned_text_USA = [lemma_pos_cleaner(txt) for txt in text_USA]
cleaned_text_China_USA = [lemma_pos_cleaner(txt) for txt in text_China_USA]

print('China:')
print(cleaned_text_China[0:10])
print()
print('USA:')
print(cleaned_text_USA[0:10])
print()
print('China&USA:')
print(cleaned_text_China_USA[0:10])

China:
['world health organization express concern rise rate new covid infection death third wave virus sweep across world', 'director general world health organization warn rise global covid case death press conference emphasize public follow covid health guideline contain virus', 'vast majority covid vaccine administer far wealthy nation world health organization report', 'africa administer percent million covid vaccine dos give globally world health organization say', 'causal link covid vaccine rare occurrence blood clot low platelet consider plausible confirm world health organization say', 'mark annual world health day year call fairer healthy world everyone eliminate health inequity pandemic push people poverty food insecurity social problem', 'covid19 vaccine produce expect review emergency use end say accord people daily', 'region currently fight resurgence covid transmission world health organization regional office call upon government region scale vaccine production vaccinat

In [217]:
freq_dict_China = frequency_dictionary(cleaned_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_USA = frequency_dictionary(cleaned_text_USA)
freq_dict_USA = dict(sorted(freq_dict_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_China_USA = frequency_dictionary(cleaned_text_China_USA)
freq_dict_China_USA = dict(sorted(freq_dict_China_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

# number of words in the cleaned tweets:
print('China: ', len(list(freq_dict_China)))
print('USA: ', len(list(freq_dict_USA)))
print('China&USA: ', len(list(freq_dict_China_USA)))

China:  2005
USA:  3334
China&USA:  3860


In [218]:
# Most frequent words
print('China')
print([key for key in freq_dict_China.keys() if freq_dict_China[key]>200])
print()
print('USA')
print([key for key in freq_dict_USA.keys() if freq_dict_USA[key]>400])
print()
print('China&USA')
print([key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]>600])

China
['covid', 'vaccine', 'china']

USA
['covid', 'vaccine', 'say', 'new']

China&USA
['covid', 'vaccine', 'say']


# less frequent words

In [219]:
# less frequent words:
print('Less frequent China: ', len([key for key in freq_dict_China.keys() if freq_dict_China[key]<5]))
print('More frequent China: ', len([key for key in freq_dict_China.keys() if freq_dict_China[key]>=5]))
less_frequent_words_China = [key for key in freq_dict_China.keys() if freq_dict_China[key]<5]

print('Less frequent USA: ', len([key for key in freq_dict_USA.keys() if freq_dict_USA[key]<5]))
print('More frequent USA:', len([key for key in freq_dict_USA.keys() if freq_dict_USA[key]>=5]))
less_frequent_words_USA = [key for key in freq_dict_USA.keys() if freq_dict_USA[key]<5]

print('Less frequent China&USA: ', len([key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]<5]))
print('More frequent China&USA: ', len([key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]>=5]))
less_frequent_words_China_USA = [key for key in freq_dict_China_USA.keys() if freq_dict_China_USA[key]<5]

Less frequent China:  1442
More frequent China:  563
Less frequent USA:  2094
More frequent USA: 1240
Less frequent China&USA:  2387
More frequent China&USA:  1473


In [220]:
# discard less frequent words
cleaned_mostfreq_text_China = cleaned_text_China.copy()
for txt in range(len(cleaned_mostfreq_text_China)):
  if txt % 1000 == 0:
    print(txt, '/',len(cleaned_mostfreq_text_China))
  for word in less_frequent_words_China:
    if word in cleaned_mostfreq_text_China[txt].split():
      cleaned_mostfreq_text_China[txt] = cleaned_mostfreq_text_China[txt].replace(word, '')
      cleaned_mostfreq_text_China[txt] = " ".join(cleaned_mostfreq_text_China[txt].split())

cleaned_mostfreq_text_USA = cleaned_text_USA.copy()
for txt in range(len(cleaned_mostfreq_text_USA)):
  if txt % 1000 == 0:
    print(txt, '/',len(cleaned_mostfreq_text_USA))
  for word in less_frequent_words_USA:
    if word in cleaned_mostfreq_text_USA[txt].split():
      cleaned_mostfreq_text_USA[txt] = cleaned_mostfreq_text_USA[txt].replace(word, '')
      cleaned_mostfreq_text_USA[txt] = " ".join(cleaned_mostfreq_text_USA[txt].split())

cleaned_mostfreq_text_China_USA = cleaned_text_China_USA.copy()
for txt in range(len(cleaned_mostfreq_text_China_USA)):
  if txt % 1000 == 0:
    print(txt, '/',len(cleaned_mostfreq_text_China_USA))
  for word in less_frequent_words_China_USA:
    if word in cleaned_mostfreq_text_China_USA[txt].split():
      cleaned_mostfreq_text_China_USA[txt] = cleaned_mostfreq_text_China_USA[txt].replace(word, '')
      cleaned_mostfreq_text_China_USA[txt] = " ".join(cleaned_mostfreq_text_China_USA[txt].split())

0 / 1336
1000 / 1336
0 / 3957
1000 / 3957
2000 / 3957
3000 / 3957
0 / 5293
1000 / 5293
2000 / 5293
3000 / 5293
4000 / 5293
5000 / 5293


In [221]:
freq_dict_China = frequency_dictionary(cleaned_mostfreq_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_USA = frequency_dictionary(cleaned_mostfreq_text_USA)
freq_dict_USA = dict(sorted(freq_dict_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_China_USA = frequency_dictionary(cleaned_mostfreq_text_China_USA)
freq_dict_China_USA = dict(sorted(freq_dict_China_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

# number of words in the cleaned tweets:
print('China: ', len(list(freq_dict_China)))
print('USA: ', len(list(freq_dict_USA)))
print('China&USA: ', len(list(freq_dict_China_USA)))

China:  568
USA:  1246
China&USA:  1484


# Build Network

In [222]:
df_China = pd.DataFrame.from_dict(freq_dict_China, orient='index').reset_index()
df_China.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China.sort_values(by=['Count'], ascending=False, inplace=True)
df_China.reset_index(inplace=True)
df_China.drop(columns="index",inplace=True)

df_USA = pd.DataFrame.from_dict(freq_dict_USA, orient='index').reset_index()
df_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_USA.reset_index(inplace=True)
df_USA.drop(columns="index",inplace=True)

df_China_USA = pd.DataFrame.from_dict(freq_dict_China_USA, orient='index').reset_index()
df_China_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_China_USA.reset_index(inplace=True)
df_China_USA.drop(columns="index",inplace=True)

print('China')
print(df_China.iloc[0:30])
print()
print('USA')
print(df_USA.iloc[0:30])
print()
print('China&USA')
print(df_China_USA.iloc[0:30])
print()

China
           Word  Count
0         covid    959
1       vaccine    767
2         china    328
3           say    191
4        report    175
5        health    173
6          case    172
7       country    172
8       receive    159
9         first    156
10      million    151
11          new    150
12          dos    123
13       people    120
14     pandemic    113
15        batch    105
16        world     96
17    president     77
18       accord     72
19         dose     71
20          use     70
21     national     67
22        death     65
23          day     65
24  vaccination     65
25          one     64
26         year     63
27       brazil     63
28       second     59
29   administer     59

USA
           Word  Count
0         covid   3021
1       vaccine   2128
2           say    879
3           new    413
4          case    395
5       johnson    298
6          shot    259
7       million    258
8        report    253
9           use    253
10       people    247


In [223]:
keys_China = freq_dict_China.keys()  
keys_USA = freq_dict_USA.keys()  
keys_China_USA = freq_dict_China_USA.keys()  

In [151]:
def create_network(cleaned_text):
  network = {}
  #connect the word that appear in the same tweets
  for row in cleaned_text:
    combined_list = [word for word in str.split(row)]
    #for pair in itertools.product(combined_list, combined_list):
    #print(combined_list)
    for pair in itertools.product(combined_list, combined_list):
          #exclude self-loops and count each pair only once because our graph is undirected and we do not take self-loops into account
          if pair[0]!=pair[1] and not(pair[::-1] in network):
              network.setdefault(pair,0)
              network[pair] += 1 
  network_df = pd.DataFrame.from_dict(network, orient="index")
  network_df.columns = ["weight"]
  network_df.sort_values(by="weight",inplace=True, ascending=False)
  return network, network_df

In [224]:
network_China, network_df_China = create_network(cleaned_mostfreq_text_China)
network_USA, network_df_USA = create_network(cleaned_mostfreq_text_USA)
network_China_USA, network_df_China_USA = create_network(cleaned_mostfreq_text_China_USA)

In [225]:
print('China:')
print(network_df_China.iloc[0:30])
print()
print('USA:')
print(network_df_USA.iloc[0:30])
print()
print('China&USA:')
print(network_df_China_USA.iloc[0:30])
print()

China:
                    weight
(covid, vaccine)       529
(vaccine, china)       253
(china, covid)         249
(receive, vaccine)     171
(covid, case)          143
(million, covid)       141
(health, covid)        139
(vaccine, say)         137
(covid, say)           135
(vaccine, dos)         135
(country, vaccine)     134
(covid, country)       127
(new, covid)           125
(first, vaccine)       125
(covid, report)        121
(first, covid)         119
(receive, covid)       118
(million, vaccine)     113
(vaccine, health)      110
(batch, vaccine)       109
(covid, dos)           105
(covid, pandemic)      103
(new, case)             98
(covid, people)         92
(batch, covid)          90
(million, dos)          90
(vaccine, use)          83
(case, report)          81
(world, covid)          74
(dose, vaccine)         73

USA:
                      weight
(vaccine, covid)        1459
(covid, say)             649
(vaccine, say)           633
(new, covid)             363
(covi

#Graph


In [193]:
def get_graph(network):
  up_weighted = []
  for edge in network:
      #we can filter edges by weight by uncommenting the next line and setting desired weight threshold
      up_weighted.append((edge[0],edge[1],network[edge]))
      
  #print(network)
  #print(up_weighted[0:10])
  G = nx.Graph()
  G.add_weighted_edges_from(up_weighted)
  return G

In [226]:
G_China = get_graph(network_China)
G_USA = get_graph(network_USA)
G_China_USA = get_graph(network_China_USA)

In [227]:
print('China:')
print('Nodes: ',len(G_China.nodes()))
print('Edges: ',len(G_China.edges()))
print('Is connected: ',nx.is_connected(G_China))
print()
print('USA:')
print('Nodes: ',len(G_USA.nodes()))
print('Edges: ',len(G_USA.edges()))
print('Is connected: ',nx.is_connected(G_USA))
print()
print('China&USA:')
print('Nodes: ',len(G_China_USA.nodes()))
print('Edges: ',len(G_China_USA.edges()))
print('Is connected: ',nx.is_connected(G_China_USA))

China:
Nodes:  568
Edges:  19887
Is connected:  True

USA:
Nodes:  1246
Edges:  63970
Is connected:  True

China&USA:
Nodes:  1484
Edges:  87741
Is connected:  True


# PageRank

In [228]:
# Calculating the pagerank on graph G, teleportation probability here is 0.15 but since the graph is strongly connected we can set it to zero if we want
pr_China = nx.algorithms.pagerank(G_China,alpha = 1)
pr_China = dict(sorted(pr_China.items(), key=lambda item: item[1],reverse  = True))

pr_USA = nx.algorithms.pagerank(G_USA,alpha = 1)
pr_USA = dict(sorted(pr_USA.items(), key=lambda item: item[1],reverse  = True))

pr_China_USA = nx.algorithms.pagerank(G_China_USA,alpha = 1)
pr_China_USA = dict(sorted(pr_China_USA.items(), key=lambda item: item[1],reverse  = True))

In [197]:
def threshold(vector,threshold):

  l = [(el,vector[el]) for el in vector if vector[el] >= threshold ]

  return pd.DataFrame(l)

In [159]:
def threshold_reverse(vector,threshold):

  l = [(el,vector[el]) for el in vector if vector[el] < threshold ]

  return pd.DataFrame(l)

In [229]:
print('China: ', len(threshold(pr_China,0.0003)))
print()
print('first:')
print(threshold(pr_China,0.0003).iloc[:30])
print()
print('last:')
print(threshold(pr_China,0.0003).iloc[len(threshold(pr_China,0.0003))-30:])
print()
print('USA: ', len(threshold(pr_USA,0.0003)))
print()
print('first:')
print(threshold(pr_USA,0.0003).iloc[:30])
print()
print('last:')
print(threshold(pr_USA,0.0003).iloc[len(threshold(pr_USA,0.0003))-30:])
print()
print('China&USA: ', len(threshold(pr_China_USA,0.0003)))
print()
print('first:')
print(threshold(pr_China_USA,0.0003).iloc[:30])
print()
print('last:')
print(threshold(pr_China_USA,0.0003).iloc[len(threshold(pr_China_USA,0.0003))-30:])

China:  552

first:
               0         1
0          covid  0.070439
1        vaccine  0.055125
2          china  0.026009
3            say  0.017835
4         health  0.016636
5        country  0.015820
6         report  0.015076
7           case  0.013800
8        million  0.012868
9          first  0.012644
10       receive  0.011891
11           new  0.011646
12           dos  0.010449
13        people  0.010419
14         world  0.009165
15      pandemic  0.008211
16        accord  0.007000
17         batch  0.006803
18      national  0.006340
19           day  0.006168
20         death  0.005893
21           use  0.005830
22     president  0.005761
23   vaccination  0.005744
24        brazil  0.005573
25          year  0.005367
26          dose  0.005337
27    administer  0.005242
28  organization  0.005153
29        origin  0.005108

last:
               0         1
522     business  0.000380
523         slow  0.000380
524  development  0.000380
525         bank  0.000380
5

# TF-IDF

In [61]:
tfidf = TfidfVectorizer(ngram_range=(1,1))   # ngram range can be changed to obtain measures regarding n grams instead of single words

X_China = tfidf.fit_transform(cleaned_mostfreq_text_China).toarray()    # entry (i,j) if Tfidf measure of word_list[j] in document i
word_list_China = tfidf.get_feature_names_out()

X_USA = tfidf.fit_transform(cleaned_mostfreq_text_USA).toarray()
word_list_USA = tfidf.get_feature_names_out()

X_China_USA = tfidf.fit_transform(cleaned_mostfreq_text_China_USA).toarray()
word_list_China_USA = tfidf.get_feature_names_out()


In [62]:
tfidf_df_China = pd.DataFrame(X_China,columns = word_list_China)

tfidf_df_USA = pd.DataFrame(X_USA,columns = word_list_USA)

tfidf_df_China_USA = pd.DataFrame(X_China_USA,columns = word_list_China_USA)

In [63]:
tfidf_word_measure_China = np.mean(tfidf_df_China,axis = 0)
tfidf_word_measure_China = tfidf_word_measure_China.sort_values(ascending = False)
tfidf_word_measure_USA = np.mean(tfidf_df_USA,axis = 0)
tfidf_word_measure_USA = tfidf_word_measure_USA.sort_values(ascending = False)
tfidf_word_measure_China_USA = np.mean(tfidf_df_China_USA,axis = 0)
tfidf_word_measure_China_USA = tfidf_word_measure_China_USA.sort_values(ascending = False)

In [64]:
print('China:')
print(tfidf_word_measure_China[0:30])
print()
print('USA:')
print(tfidf_word_measure_USA[0:30])
print()
print('China&USA:')
print(tfidf_word_measure_China_USA[0:30])
print()

China:
china          0.057393
covid          0.032344
case           0.029077
novel          0.027626
coronavirus    0.027115
new            0.025339
vaccine        0.022859
hospital       0.022712
say            0.020808
outbreak       0.019470
report         0.017295
fight          0.016957
patient        0.016629
people         0.015936
confirm        0.014500
watch          0.014347
health         0.014288
country        0.014000
day            0.013577
live           0.013453
province       0.012905
first          0.012837
city           0.012793
death          0.012317
year           0.012240
world          0.011623
epidemic       0.011616
amid           0.011082
medical        0.011039
president      0.010959
dtype: float64

USA:
coronavirus    0.049972
covid          0.048809
china          0.041930
vaccine        0.031179
case           0.028555
say            0.027612
new            0.026551
report         0.017860
death          0.017719
outbreak       0.017477
virus       

# reduced graph

In [230]:
# less frequent words:
less_important_words_China = [key for key in list(threshold_reverse(pr_China,0.0003)[0])]

less_important_words_USA = [key for key in list(threshold_reverse(pr_USA,0.0003)[0])]

less_important_words_China_USA = [key for key in list(threshold_reverse(pr_China_USA,0.0003)[0])]

In [231]:
# discard less frequent words
cleaned_mostimp_text_China = cleaned_mostfreq_text_China.copy()
for txt in range(len(cleaned_mostimp_text_China)):
  if txt % 1000 == 0:
    print(txt, '/',len(cleaned_mostimp_text_China))
  for word in less_important_words_China:
    if word in cleaned_mostimp_text_China[txt].split():
      cleaned_mostimp_text_China[txt] = cleaned_mostimp_text_China[txt].replace(word, '')
      cleaned_mostimp_text_China[txt] = " ".join(cleaned_mostimp_text_China[txt].split())

cleaned_mostimp_text_USA = cleaned_mostfreq_text_USA.copy()
for txt in range(len(cleaned_mostimp_text_USA)):
  if txt % 1000 == 0:
    print(txt, '/',len(cleaned_mostimp_text_USA))
  for word in less_important_words_USA:
    if word in cleaned_mostimp_text_USA[txt].split():
      cleaned_mostimp_text_USA[txt] = cleaned_mostimp_text_USA[txt].replace(word, '')
      cleaned_mostimp_text_USA[txt] = " ".join(cleaned_mostimp_text_USA[txt].split())

cleaned_mostimp_text_China_USA = cleaned_mostfreq_text_China_USA.copy()
for txt in range(len(cleaned_mostimp_text_China_USA)):
  if txt % 1000 == 0:
    print(txt, '/',len(cleaned_mostimp_text_China_USA))
  for word in less_important_words_China_USA:
    if word in cleaned_mostimp_text_China_USA[txt].split():
      cleaned_mostimp_text_China_USA[txt] = cleaned_mostimp_text_China_USA[txt].replace(word, '')
      cleaned_mostimp_text_China_USA[txt] = " ".join(cleaned_mostimp_text_China_USA[txt].split())

0 / 1336
1000 / 1336
0 / 3957
1000 / 3957
2000 / 3957
3000 / 3957
0 / 5293
1000 / 5293
2000 / 5293
3000 / 5293
4000 / 5293
5000 / 5293


In [232]:
freq_dict_China = frequency_dictionary(cleaned_mostimp_text_China)
freq_dict_China = dict(sorted(freq_dict_China.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_USA = frequency_dictionary(cleaned_mostimp_text_USA)
freq_dict_USA = dict(sorted(freq_dict_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

freq_dict_China_USA = frequency_dictionary(cleaned_mostimp_text_China_USA)
freq_dict_China_USA = dict(sorted(freq_dict_China_USA.items(), key=lambda item: item[1], reverse = True))   #order from more frequent to less frequent word

# number of words in the cleaned tweets:
print('China: ', len(list(freq_dict_China)))
print('USA: ', len(list(freq_dict_USA)))
print('China&USA: ', len(list(freq_dict_China_USA)))

China:  556
USA:  686
China&USA:  677


In [233]:
df_China = pd.DataFrame.from_dict(freq_dict_China, orient='index').reset_index()
df_China.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China.sort_values(by=['Count'], ascending=False, inplace=True)
df_China.reset_index(inplace=True)
df_China.drop(columns="index",inplace=True)

df_USA = pd.DataFrame.from_dict(freq_dict_USA, orient='index').reset_index()
df_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_USA.reset_index(inplace=True)
df_USA.drop(columns="index",inplace=True)

df_China_USA = pd.DataFrame.from_dict(freq_dict_China_USA, orient='index').reset_index()
df_China_USA.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
df_China_USA.sort_values(by=['Count'], ascending=False, inplace=True)
df_China_USA.reset_index(inplace=True)
df_China_USA.drop(columns="index",inplace=True)

print('China')
print(df_China.iloc[0:30])
print()
print('USA')
print(df_USA.iloc[0:30])
print()
print('China&USA')
print(df_China_USA.iloc[0:30])
print()

China
           Word  Count
0         covid    959
1       vaccine    766
2         china    328
3           say    190
4        report    175
5        health    173
6          case    172
7       country    172
8       receive    159
9         first    156
10      million    151
11          new    150
12          dos    123
13       people    120
14     pandemic    113
15        batch    105
16        world     96
17    president     77
18       accord     72
19         dose     71
20          use     70
21     national     67
22        death     65
23          day     65
24  vaccination     65
25          one     64
26         year     63
27       brazil     63
28   administer     59
29       second     59

USA
           Word  Count
0         covid   3021
1       vaccine   2128
2           say    878
3           new    413
4          case    395
5       johnson    298
6          shot    259
7       million    258
8        report    253
9           use    253
10       people    247


In [234]:
keys_China = freq_dict_China.keys()  
keys_USA = freq_dict_USA.keys()  
keys_China_USA = freq_dict_China_USA.keys()  

In [235]:
network_China, network_df_China = create_network(cleaned_mostimp_text_China)
network_USA, network_df_USA = create_network(cleaned_mostimp_text_USA)
network_China_USA, network_df_China_USA = create_network(cleaned_mostimp_text_China_USA)

In [236]:
print('China:')
print(network_df_China.iloc[0:30])
print()
print('USA:')
print(network_df_USA.iloc[0:30])
print()
print('China&USA:')
print(network_df_China_USA.iloc[0:30])
print()

China:
                    weight
(covid, vaccine)       528
(vaccine, china)       253
(china, covid)         249
(receive, vaccine)     171
(covid, case)          143
(million, covid)       141
(health, covid)        139
(vaccine, say)         136
(vaccine, dos)         135
(covid, say)           134
(country, vaccine)     134
(covid, country)       127
(first, vaccine)       125
(new, covid)           125
(covid, report)        121
(first, covid)         119
(receive, covid)       118
(million, vaccine)     113
(vaccine, health)      110
(batch, vaccine)       109
(covid, dos)           105
(covid, pandemic)      103
(new, case)             98
(covid, people)         92
(batch, covid)          90
(million, dos)          90
(vaccine, use)          82
(case, report)          81
(world, covid)          74
(dose, vaccine)         73

USA:
                      weight
(vaccine, covid)        1459
(covid, say)             649
(vaccine, say)           633
(new, covid)             363
(covi

In [206]:
def get_graph(network):
  up_weighted = []
  for edge in network:
      #we can filter edges by weight by uncommenting the next line and setting desired weight threshold
      up_weighted.append((edge[0],edge[1],network[edge]))
      
  #print(network)
  #print(up_weighted[0:10])
  G = nx.Graph()
  G.add_weighted_edges_from(up_weighted)
  return G

In [237]:
G_China = get_graph(network_China)
G_USA = get_graph(network_USA)
G_China_USA = get_graph(network_China_USA)

In [238]:
print('China:')
print('Nodes: ',len(G_China.nodes()))
print('Edges: ',len(G_China.edges()))
print('Is connected: ',nx.is_connected(G_China))
print()
print('USA:')
print('Nodes: ',len(G_USA.nodes()))
print('Edges: ',len(G_USA.edges()))
print('Is connected: ',nx.is_connected(G_USA))
print()
print('China&USA:')
print('Nodes: ',len(G_China_USA.nodes()))
print('Edges: ',len(G_China_USA.edges()))
print('Is connected: ',nx.is_connected(G_China_USA))

China:
Nodes:  556
Edges:  19649
Is connected:  True

USA:
Nodes:  686
Edges:  43779
Is connected:  True

China&USA:
Nodes:  677
Edges:  52248
Is connected:  True


#Save edge list

In [239]:
filename = './edgelist_China_3_700.csv'
nx.write_weighted_edgelist(G_China, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_China_3_700.csv
files.download("edgelist_China_3_700.csv")

filename = './edgelist_USA_3_700.csv'
nx.write_weighted_edgelist(G_USA, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_USA_3_700.csv
files.download("edgelist_USA_3_700.csv")

filename = './edgelist_China_USA_3_700.csv'
nx.write_weighted_edgelist(G_China_USA, filename, delimiter=",")
#add header with appropriate column names (works on colab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./edgelist_China_USA_3_700.csv
files.download("edgelist_China_USA_3_700.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>





# Create Node List




In [125]:
def nodes(freq_dict, name):
  word_nodes = pd.DataFrame.from_dict(freq_dict,orient="index")
  word_nodes.reset_index(inplace=True)
  word_nodes["Label"] = word_nodes["index"]
  word_nodes.rename(columns={"index":"Id",0:"delete"},inplace=True)
  word_nodes = word_nodes.drop(columns=['delete'])
  nodelist = pd.DataFrame()
  nodelist = nodelist.append(word_nodes, ignore_index=True)

  nodelist = nodelist.to_csv("nodelist_"+name+".csv",index=False)
  files.download("nodelist_"+name+".csv")
  return nodelist, word_nodes

In [240]:
nodelist_China, word_nodes_China = nodes(freq_dict_China,"China_3_700")
nodelist_USA, word_nodes_USA = nodes(freq_dict_USA,"USA_3_700")
nodelist_China_USA, word_nodes_China_USA = nodes(freq_dict_China_USA,"China_3_USA_700")

print('China:')
print(word_nodes_China.head())
print()
print('USA:')
print(word_nodes_USA.head())
print()
print('China&USA:')
print(word_nodes_China_USA.head())
print()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

China:
        Id    Label
0    covid    covid
1  vaccine  vaccine
2    china    china
3      say      say
4   report   report

USA:
        Id    Label
0    covid    covid
1  vaccine  vaccine
2      say      say
3      new      new
4     case     case

China&USA:
        Id    Label
0    covid    covid
1  vaccine  vaccine
2      say      say
3     case     case
4      new      new

